In [ ]:
import pandas as pd  # a module which provides the data structures and functions to store and manipulate tables in dataframes
import pydbtools as pydb  # A module which allows SQL queries to be run on the Analytical Platform from Python, see https://github.com/moj-analytical-services/pydbtools
import boto3  # allows you to directly create, update, and delete AWS resources from Python scripts

# sets parameters to view dataframes for tables easier
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [ ]:
#Variables to be used in this notebook

#this is the athena database we will be storing our tables in
fcsq_database = "fcsq"

#this is the s3 bucket we will be saving data to
s3 = boto3.resource("s3")
bucket = s3.Bucket("alpha-family-data")

#Last full year to be published - including this publication
annual_year = 2023

#Current publication variables
current_year = 2024
current_quarter = 2

#Next publication variables
next_quarter_year = 2024 
next_quarter = 3

In [ ]:
# All timeliness calculations

In [ ]:
#Petitions - annual, with digital/paper split, with law split - averages table

create_averages_1_table =f"""
SELECT
  CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
       ELSE new_case_type 
  END AS "Case",
  Stage,
  Year,
  CAST(NULL AS VARCHAR) AS Quarter,
  digital_paper as case_type,
  law,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM fcsq.divorce_fct_petitions

GROUP BY
  CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
       ELSE new_case_type 
       END,
  Stage,
  Year,
  digital_paper,
  law;
"""
pydb.create_temp_table(create_averages_1_table,'averages_1')

In [ ]:
#Petitions - annual, with digital/paper split without law split - averages table

create_averages_2_table =f"""
SELECT
  CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
       ELSE new_case_type 
  END AS "Case",
  Stage,
  Year,
  CAST(NULL AS VARCHAR) AS Quarter,
  digital_paper as case_type,
  'All' as law,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM fcsq.divorce_fct_petitions

GROUP BY
  CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
       ELSE new_case_type 
       END,
  Stage,
  Year,
  digital_paper;
"""
pydb.create_temp_table(create_averages_2_table,'averages_2')

In [ ]:
#Petitions - annual, without digital/paper split, with law split - averages table

create_averages_3_table =f"""
SELECT
  CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
       ELSE new_case_type 
  END AS "Case",
  Stage,
  Year,
  CAST(NULL AS VARCHAR) AS Quarter,
  'All' as case_type,
  law,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM fcsq.divorce_fct_petitions

GROUP BY
  CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
       ELSE new_case_type 
       END,
  Stage,
  Year,
  law;
"""
pydb.create_temp_table(create_averages_3_table,'averages_3')

In [ ]:
#Petitions - annual, without digital/paper split, without law split - averages table

create_averages_4_table =f"""
SELECT
  CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
       ELSE new_case_type 
  END AS "Case",
  Stage,
  Year,
  CAST(NULL AS VARCHAR) AS Quarter,
  'All' as case_type,
  'All' as law,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM fcsq.divorce_fct_petitions

GROUP BY
  CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
       ELSE new_case_type 
       END,
  Stage,
  Year;
"""
pydb.create_temp_table(create_averages_4_table,'averages_4')

In [ ]:
#Petitions - Quarterly, with digital/paper split, with law split - averages table

create_averages_5_table =f"""
SELECT
  CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
       ELSE new_case_type 
  END AS "Case",
  Stage,
  CAST(NULL AS INT) AS Year,
  CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
  digital_paper as case_type,
  law,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM  fcsq.divorce_fct_petitions

GROUP BY
  CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
       ELSE new_case_type 
       END,
  Stage,
  Year,
  Quarter,
  digital_paper,
  law; 
"""
pydb.create_temp_table(create_averages_5_table,'averages_5')

In [ ]:
#Petitions - Quarterly, with digital/paper split without law split - averages table

create_averages_6_table =f"""
SELECT
  CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
       ELSE new_case_type 
  END AS "Case",
  Stage,
  CAST(NULL AS INT) AS Year,
  CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
  digital_paper as case_type,
  'All' as law,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM fcsq.divorce_fct_petitions

GROUP BY
  CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
       ELSE new_case_type 
       END,
  Stage,
  Year,
  Quarter,
  digital_paper;
"""
pydb.create_temp_table(create_averages_6_table,'averages_6')

In [ ]:
#Petitions - Quarterly, without digital/paper split, with law split - averages table

create_averages_7_table =f"""
SELECT
  CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
       ELSE new_case_type 
  END AS "Case",
  Stage,
  CAST(NULL AS INT) AS Year,
  CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
  'All' as case_type,
  law,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM fcsq.divorce_fct_petitions
GROUP BY
  CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
       ELSE new_case_type 
       END,
  Stage,
  Year,
  Quarter,
  law; 
"""
pydb.create_temp_table(create_averages_7_table,'averages_7')

In [ ]:
#Petitions - Quarterly, without digital/paper split, without law split - averages table

create_averages_8_table =f"""
SELECT
  CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
       ELSE new_case_type 
  END AS "Case",
  Stage,
  CAST(NULL AS INT) AS Year,
  CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
  'All' as case_type,
  'All' as law,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM fcsq.divorce_fct_petitions

GROUP BY
  CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
       ELSE new_case_type 
       END,
  Stage,
  Year,
  Quarter;
"""
pydb.create_temp_table(create_averages_8_table,'averages_8')

In [ ]:
# Nisi - Annual, with digital/paper split, with law split - averages table

create_averages_9_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  case_type,
  law,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_nisi >= 0 THEN pet_to_nisi END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END) + MIN(CASE WHEN data_half = 2 THEN pet_to_nisi END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
         ELSE new_case_type 
    END AS "Case",
    Stage,
    Year,
    CAST(NULL AS VARCHAR) AS Quarter,
    digital_paper AS case_type,
    law,
    pet_to_nisi,
    NTILE(2) OVER (PARTITION BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, year, digital_paper, law 
                      ORDER BY pet_to_nisi) AS data_half

  FROM fcsq.divorce_fct_nisi

  ORDER BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, year, digital_paper, law, pet_to_nisi) AS t1
 
GROUP BY "Case", Stage, Year, Quarter, case_type, law

ORDER BY "Case", Stage, Year, Quarter, case_type, law;
"""
pydb.create_temp_table(create_averages_9_table,'averages_9')

In [ ]:
#Nisi - Annual, with digital/paper split, without law split - averages table

create_averages_10_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  case_type,
  law,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_nisi >= 0 THEN pet_to_nisi END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END) + MIN(CASE WHEN data_half = 2 THEN pet_to_nisi END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
         ELSE new_case_type 
    END AS "Case",
    Stage,
    Year,
    CAST(NULL AS VARCHAR) AS Quarter,
    digital_paper AS case_type,
    'All' AS law,
    pet_to_nisi,
    NTILE(2) OVER (PARTITION BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, year, digital_paper 
                   ORDER BY pet_to_nisi) AS data_half

  FROM fcsq.divorce_fct_nisi

  ORDER BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, year, digital_paper, pet_to_nisi) AS t2
 
GROUP BY "Case", Stage, Year, Quarter, case_type, law

ORDER BY "Case", Stage, Year, Quarter, case_type, law;
"""
pydb.create_temp_table(create_averages_10_table,'averages_10')

In [ ]:
#Nisi - Annual, without digital/paper split, with law split - averages table

create_averages_11_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  case_type,
  law,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_nisi >= 0 THEN pet_to_nisi END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END) + MIN(CASE WHEN data_half = 2 THEN pet_to_nisi END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
         ELSE new_case_type 
    END AS "Case",
    Stage,
    Year,
    CAST(NULL AS VARCHAR) AS Quarter,
    'All' AS case_type,
    law,
    pet_to_nisi,
    NTILE(2) OVER (PARTITION BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, year, law 
                      ORDER BY pet_to_nisi) AS data_half

  FROM fcsq.divorce_fct_nisi

  ORDER BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, year, law, pet_to_nisi) AS t3
 
GROUP BY "Case", Stage, Year, Quarter, case_type, law

ORDER BY "Case", Stage, Year, Quarter, case_type, law;
"""
pydb.create_temp_table(create_averages_11_table,'averages_11')

In [ ]:
#Nisi - Annual, without digital/paper split, without law split - averages table

create_averages_12_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  case_type,
  law,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_nisi >= 0 THEN pet_to_nisi END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END) + MIN(CASE WHEN data_half = 2 THEN pet_to_nisi END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
         ELSE new_case_type 
    END AS "Case",
    Stage,
    Year,
    CAST(NULL AS VARCHAR) AS Quarter,
    'All' AS case_type,
    'All' AS law,
    pet_to_nisi,
    NTILE(2) OVER (PARTITION BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, year 
                      ORDER BY pet_to_nisi) AS data_half

  FROM fcsq.divorce_fct_nisi

  ORDER BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, year, pet_to_nisi) AS t4
 
GROUP BY "Case", Stage, Year, Quarter, case_type, law

ORDER BY "Case", Stage, Year, Quarter, case_type, law;
"""
pydb.create_temp_table(create_averages_12_table,'averages_12')

In [ ]:
#Nisi - Quarterly, with digital/paper split, with law split - averages table 

create_averages_13_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  case_type,
  law,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_nisi >= 0 THEN pet_to_nisi END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END) + MIN(CASE WHEN data_half = 2 THEN pet_to_nisi END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
         ELSE new_case_type 
    END AS "Case",
    Stage,
    CAST(NULL AS INT) AS Year,
    CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
    digital_paper AS case_type,
    law,
    pet_to_nisi,
    NTILE(2) OVER (PARTITION BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)), digital_paper, law 
                      ORDER BY pet_to_nisi) AS data_half

  FROM fcsq.divorce_fct_nisi

  ORDER BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)), digital_paper, law, pet_to_nisi) AS t5
 
GROUP BY "Case", Stage, Year, Quarter, case_type, law

ORDER BY "Case", Stage, Year, Quarter, case_type, law;
"""
pydb.create_temp_table(create_averages_13_table,'averages_13')

In [ ]:
#Nisi - Quarterly, with digital/paper split, without law split - averages table

create_averages_14_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  case_type,
  law,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_nisi >= 0 THEN pet_to_nisi END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END) + MIN(CASE WHEN data_half = 2 THEN pet_to_nisi END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
         ELSE new_case_type 
    END AS "Case",
    Stage,
    CAST(NULL AS INT) AS Year,
    CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
    digital_paper AS case_type,
    'All' AS law,
    pet_to_nisi,
    NTILE(2) OVER (PARTITION BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)), digital_paper 
                      ORDER BY pet_to_nisi) AS data_half

  FROM fcsq.divorce_fct_nisi

  ORDER BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)), digital_paper, pet_to_nisi) AS t6
 
GROUP BY "Case", Stage, Year, Quarter, case_type, law

ORDER BY "Case", Stage, Year, Quarter, case_type, law;
"""
pydb.create_temp_table(create_averages_14_table,'averages_14')

In [ ]:
#Nisi - Quarterly, without digital/paper split, with law split - averages table

create_averages_15_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  case_type,
  law,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_nisi >= 0 THEN pet_to_nisi END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END) + MIN(CASE WHEN data_half = 2 THEN pet_to_nisi END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
         ELSE new_case_type 
    END AS "Case",
    Stage,
    CAST(NULL AS INT) AS Year,
    CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
    'All' AS case_type,
    law,
    pet_to_nisi,
    NTILE(2) OVER (PARTITION BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)), law 
                      ORDER BY pet_to_nisi) AS data_half

  FROM fcsq.divorce_fct_nisi

  ORDER BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)), law, pet_to_nisi) AS t7
 
GROUP BY "Case", Stage, Year, Quarter, case_type, law

ORDER BY "Case", Stage, Year, Quarter, case_type, law;
"""
pydb.create_temp_table(create_averages_15_table,'averages_15')

In [ ]:
#Nisi - Quarterly, without digital/paper split, without law split - averages table

create_averages_16_table = f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  case_type,
  law,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_nisi >= 0 THEN pet_to_nisi END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END) + MIN(CASE WHEN data_half = 2 THEN pet_to_nisi END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
         ELSE new_case_type 
    END AS "Case",
    Stage,
    CAST(NULL AS INT) AS Year,
    CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
    'All' AS case_type,
    'All' AS law,
    pet_to_nisi,
    NTILE(2) OVER (PARTITION BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) 
                      ORDER BY pet_to_nisi) AS data_half

  FROM fcsq.divorce_fct_nisi

  ORDER BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)), pet_to_nisi) AS t8
 
GROUP BY "Case", Stage, Year, Quarter, case_type, law

ORDER BY "Case", Stage, Year, Quarter, case_type, law;
"""
pydb.create_temp_table(create_averages_16_table,'averages_16')

In [ ]:
#Absolute - Annual, with digital/paper split, with law split - averages table

create_averages_17_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  case_type,
  law,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_abs >= 0 THEN pet_to_abs END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_abs END) + MIN(CASE WHEN data_half = 2 THEN pet_to_abs END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_abs END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
         ELSE new_case_type 
    END AS "Case",
    Stage,
    Year,
    CAST(NULL AS VARCHAR) AS Quarter,
    digital_paper AS case_type,
    law,
    pet_to_abs,
    NTILE(2) OVER (PARTITION BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, year, digital_paper, law 
                      ORDER BY pet_to_abs) AS data_half

  FROM fcsq.divorce_fct_abs

  ORDER BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, year, digital_paper, law, pet_to_abs) AS t9

GROUP BY "Case", Stage, Year, Quarter, case_type, law

ORDER BY "Case", Stage, Year, Quarter, case_type, law;
"""
pydb.create_temp_table(create_averages_17_table,'averages_17')

In [ ]:
#Absolute - Annual, with digital/paper split, without law split - averages table

create_averages_18_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  case_type,
  law,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_abs >= 0 THEN pet_to_abs END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_abs END) + MIN(CASE WHEN data_half = 2 THEN pet_to_abs END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_abs END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
         ELSE new_case_type 
    END AS "Case",
    Stage,
    Year,
    CAST(NULL AS VARCHAR) AS Quarter,
    digital_paper AS case_type,
    'All' AS law,
    pet_to_abs,
    NTILE(2) OVER (PARTITION BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, year, digital_paper 
                      ORDER BY pet_to_abs) AS data_half

  FROM fcsq.divorce_fct_abs

  ORDER BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, year, digital_paper, pet_to_abs) AS t10

GROUP BY "Case", Stage, Year, Quarter, case_type, law

ORDER BY "Case", Stage, Year, Quarter, case_type, law;
"""
pydb.create_temp_table(create_averages_18_table,'averages_18')

In [ ]:
#Absolute - Annual, without digital/paper split, with law split - averages table

create_averages_19_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  case_type,
  law,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_abs >= 0 THEN pet_to_abs END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_abs END) + MIN(CASE WHEN data_half = 2 THEN pet_to_abs END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_abs END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
         ELSE new_case_type 
    END AS "Case",
    Stage,
    Year,
    CAST(NULL AS VARCHAR) AS Quarter,
    'All' AS case_type,
    law,
    pet_to_abs,
    NTILE(2) OVER (PARTITION BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, year, law 
                      ORDER BY pet_to_abs) AS data_half

  FROM fcsq.divorce_fct_abs

  ORDER BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, year, law, pet_to_abs) AS t11

GROUP BY "Case", Stage, Year, Quarter, case_type, law

ORDER BY "Case", Stage, Year, Quarter, case_type, law;
"""
pydb.create_temp_table(create_averages_19_table,'averages_19')

In [ ]:
#Absolute - Annual, without digital/paper split, without law split - averages table

create_averages_20_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  case_type,
  law,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_abs >= 0 THEN pet_to_abs END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_abs END) + MIN(CASE WHEN data_half = 2 THEN pet_to_abs END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_abs END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
         ELSE new_case_type 
    END AS "Case",
    Stage,
    Year,
   CAST(NULL AS VARCHAR) AS Quarter,
    'All' AS case_type,
    'All' AS law,
    pet_to_abs,
    NTILE(2) OVER (PARTITION BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, year 
                      ORDER BY pet_to_abs) AS data_half

  FROM fcsq.divorce_fct_abs

  ORDER BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, year, pet_to_abs) AS t12

GROUP BY "Case", Stage, Year, Quarter, case_type, law

ORDER BY "Case", Stage, Year, Quarter, case_type, law;
"""
pydb.create_temp_table(create_averages_20_table,'averages_20')

In [ ]:
#Absolute - Quarterly, with digital/paper split, with law split - averages table 

create_averages_21_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  case_type,
  law,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_abs >= 0 THEN pet_to_abs END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_abs END) + MIN(CASE WHEN data_half = 2 THEN pet_to_abs END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_abs END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
         ELSE new_case_type 
    END AS "Case",
    Stage,
    CAST(NULL AS INT) AS Year,
    CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
    digital_paper AS case_type,
    law,
    pet_to_abs,
    NTILE(2) OVER (PARTITION BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)), digital_paper, law 
                      ORDER BY pet_to_abs) AS data_half

  FROM fcsq.divorce_fct_abs

  ORDER BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)), digital_paper, law, pet_to_abs) AS t13

GROUP BY "Case", Stage, Year, Quarter, case_type, law

ORDER BY "Case", Stage, Year, Quarter, case_type, law;
"""
pydb.create_temp_table(create_averages_21_table,'averages_21')

In [ ]:
#Absolute - Quarterly, with digital/paper split, without law split - averages table

create_averages_22_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  case_type,
  law,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_abs >= 0 THEN pet_to_abs END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_abs END) + MIN(CASE WHEN data_half = 2 THEN pet_to_abs END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_abs END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
         ELSE new_case_type 
    END AS "Case",
    Stage,
    CAST(NULL AS INT) AS Year,
    CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
    digital_paper AS case_type,
    'All' AS law,
    pet_to_abs,
    NTILE(2) OVER (PARTITION BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)), digital_paper 
                      ORDER BY pet_to_abs) AS data_half

  FROM fcsq.divorce_fct_abs

  ORDER BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)), digital_paper, pet_to_abs) AS t14

GROUP BY "Case", Stage, Year, Quarter, case_type, law

ORDER BY "Case", Stage, Year, Quarter, case_type, law;
"""
pydb.create_temp_table(create_averages_22_table,'averages_22')

In [ ]:
#Absolute - Quarterly, without digital/paper split, with law split - averages table

create_averages_23_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  case_type,
  law,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_abs >= 0 THEN pet_to_abs END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_abs END) + MIN(CASE WHEN data_half = 2 THEN pet_to_abs END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_abs END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
         ELSE new_case_type 
    END AS "Case",
    Stage,
    CAST(NULL AS INT) AS Year,
    CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
    'All' AS case_type,
    law,
    pet_to_abs,
    NTILE(2) OVER (PARTITION BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)), law 
                      ORDER BY pet_to_abs) AS data_half

  FROM fcsq.divorce_fct_abs

  ORDER BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)), law, pet_to_abs) AS t15

GROUP BY "Case", Stage, Year, Quarter, case_type, law

ORDER BY "Case", Stage, Year, Quarter, case_type, law;
"""
pydb.create_temp_table(create_averages_23_table,'averages_23')

In [ ]:
#Absolute - Quarterly, without digital/paper split, without law split - averages table

create_averages_24_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  case_type,
  law,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_abs >= 0 THEN pet_to_abs END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_abs END) + MIN(CASE WHEN data_half = 2 THEN pet_to_abs END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_abs END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce'
         ELSE new_case_type 
    END AS "Case",
    Stage,
    CAST(NULL AS INT) AS Year,
    CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
    'All' AS case_type,
    'All' AS law,
    pet_to_abs,
    NTILE(2) OVER (PARTITION BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) 
                      ORDER BY pet_to_abs) AS data_half

  FROM fcsq.divorce_fct_abs

  ORDER BY CASE WHEN new_case_type IN ('Divorce', 'Civil Partnership') THEN 'Divorce' ELSE new_case_type END, stage, CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)), pet_to_abs) AS t16

GROUP BY "Case", Stage, Year, Quarter, case_type, law

ORDER BY "Case", Stage, Year, Quarter, case_type, law;

"""
pydb.create_temp_table(create_averages_24_table,'averages_24')

In [ ]:
drop_divorce_timeliness_all = "DROP TABLE IF EXISTS fcsq.divorce_timeliness_all"
pydb.start_query_execution_and_wait(drop_divorce_timeliness_all)
bucket.objects.filter(Prefix="fcsq_processing/Divorce/divorce_timeliness_all").delete();

In [ ]:
#Creating timeliness table combining all the average tables together 

create_divorce_timeliness_all_table =f"""
CREATE TABLE IF NOT EXISTS fcsq.divorce_timeliness_all
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Divorce/divorce_timeliness_all') AS

SELECT * FROM __temp__.averages_1
UNION ALL
SELECT * FROM __temp__.averages_2
UNION ALL
SELECT * FROM __temp__.averages_3
UNION ALL
SELECT * FROM __temp__.averages_4
UNION ALL
SELECT * FROM __temp__.averages_5
UNION ALL
SELECT * FROM __temp__.averages_6
UNION ALL
SELECT * FROM __temp__.averages_7
UNION ALL
SELECT * FROM __temp__.averages_8
UNION ALL
SELECT * FROM __temp__.averages_9
UNION ALL
SELECT * FROM __temp__.averages_10
UNION ALL
SELECT * FROM __temp__.averages_11
UNION ALL
SELECT * FROM __temp__.averages_12
UNION ALL
SELECT * FROM __temp__.averages_13
UNION ALL
SELECT * FROM __temp__.averages_14
UNION ALL
SELECT * FROM __temp__.averages_15
UNION ALL
SELECT * FROM __temp__.averages_16
UNION ALL
SELECT * FROM __temp__.averages_17
UNION ALL
SELECT * FROM __temp__.averages_18
UNION ALL
SELECT * FROM __temp__.averages_19
UNION ALL
SELECT * FROM __temp__.averages_20
UNION ALL
SELECT * FROM __temp__.averages_21
UNION ALL
SELECT * FROM __temp__.averages_22
UNION ALL
SELECT * FROM __temp__.averages_23
UNION ALL
SELECT * FROM __temp__.averages_24
;
"""
pydb.start_query_execution_and_wait(create_divorce_timeliness_all_table)

In [ ]:
test = pydb.read_sql_query("SELECT * from fcsq.divorce_timeliness_all LIMIT 10")
test

In [ ]:
#pydb.delete_table_and_data(database="__temp__", table="divorce_timeliness_all")

In [ ]:
#New divorce law only timeliness

In [ ]:
#Applications - Annual, with applicant split - averages table

create_averages_25_table =f"""
SELECT
  New_case_type AS "Case",
  CASE WHEN Stage = 'Petition' then 'Application'
       WHEN Stage = 'Decree Nisi' then 'Conditional Order'
       WHEN Stage = 'Decree Absolute' then 'Final Order'
       ELSE 'check' 
  END AS Stage,
  Year,
  CAST(NULL AS VARCHAR) AS Quarter,
  nfd_app_type as applicant,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM fcsq.divorce_fct_petitions

WHERE law = 'New'

GROUP BY New_case_type, Stage, Year, nfd_app_type; 
"""
pydb.create_temp_table(create_averages_25_table,'averages_25')

In [ ]:
#Applications - Annual, without applicant split - averages table

create_averages_26_table =f"""
SELECT
  New_case_type AS "Case",
  CASE WHEN Stage = 'Petition' then 'Application'
       WHEN Stage = 'Decree Nisi' then 'Conditional Order'
       WHEN Stage = 'Decree Absolute' then 'Final Order'
       ELSE 'check' 
  END AS Stage,
  Year,
  CAST(NULL AS VARCHAR) AS Quarter,
  'All' as applicant,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM fcsq.divorce_fct_petitions

WHERE law = 'New'

GROUP BY New_case_type, Stage, Year;
"""
pydb.create_temp_table(create_averages_26_table,'averages_26')

In [ ]:
 #Applications - Quarterly, with applicant split - averages table 

create_averages_27_table =f"""
SELECT
  New_case_type AS "Case",
  CASE WHEN Stage = 'Petition' then 'Application'
       WHEN Stage = 'Decree Nisi' then 'Conditional Order'
       WHEN Stage = 'Decree Absolute' then 'Final Order'
       ELSE 'check' 
  END AS Stage,
  CAST(NULL AS INT) AS Year,
  CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
  nfd_app_type as applicant,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM fcsq.divorce_fct_petitions

WHERE law = 'New'

GROUP BY New_case_type, Stage, Year, Quarter, nfd_app_type; 
"""
pydb.create_temp_table(create_averages_27_table,'averages_27')


In [ ]:
#Applications - Quarterly, without applicant split - averages table

create_averages_28_table =f"""
SELECT
  New_case_type AS "Case",
  CASE WHEN Stage = 'Petition' then 'Application'
       WHEN Stage = 'Decree Nisi' then 'Conditional Order'
       WHEN Stage = 'Decree Absolute' then 'Final Order'
       ELSE 'check' 
  END AS Stage,
  CAST(NULL AS INT) AS Year,
  CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
  'All' as applicant,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM fcsq.divorce_fct_petitions

WHERE law = 'New'

GROUP BY New_case_type, Stage, Year, Quarter;
"""
pydb.create_temp_table(create_averages_28_table,'averages_28')


In [ ]:
#Conditional order - Annual, with applicant split - averages table

create_averages_29_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  applicant,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_nisi >= 0 THEN pet_to_nisi END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END) + MIN(CASE WHEN data_half = 2 THEN pet_to_nisi END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    new_case_type as "Case",
    CASE WHEN Stage = 'Petition' then 'Application'
         WHEN Stage = 'Decree Nisi' then 'Conditional Order'
         WHEN Stage = 'Decree Absolute' then 'Final Order'
         ELSE 'check' 
    END AS Stage ,
    Year,
    CAST(NULL AS VARCHAR) AS Quarter,
    nfd_app_type as applicant,
    pet_to_nisi,
    NTILE(2) OVER (PARTITION BY new_case_type, stage, year, nfd_app_type 
                   ORDER BY pet_to_nisi) AS data_half

  FROM fcsq.divorce_fct_nisi
  
  WHERE law = 'New'
  
  ORDER BY new_case_type, stage, year, nfd_app_type)
  
GROUP BY "Case", Stage, Year, Quarter, applicant

ORDER BY "Case", Stage, Year, Quarter, applicant;
"""
pydb.create_temp_table(create_averages_29_table,'averages_29')

In [ ]:
#Conditional order - Annual, without applicant split - averages table

create_averages_30_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  applicant,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_nisi >= 0 THEN pet_to_nisi END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END) + MIN(CASE WHEN data_half = 2 THEN pet_to_nisi END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    new_case_type as "Case",
    CASE WHEN Stage = 'Petition' then 'Application'
         WHEN Stage = 'Decree Nisi' then 'Conditional Order'
         WHEN Stage = 'Decree Absolute' then 'Final Order'
         ELSE 'check' 
    END AS Stage ,
    Year,
    CAST(NULL AS VARCHAR) AS Quarter,
    'All' AS applicant,
    pet_to_nisi,
    NTILE(2) OVER (PARTITION BY new_case_type, stage, year 
                   ORDER BY pet_to_nisi) AS data_half

  FROM fcsq.divorce_fct_nisi
  
  WHERE law = 'New'
  
  ORDER BY new_case_type, stage, year)
  
GROUP BY "Case", Stage, Year, Quarter, applicant

ORDER BY "Case", Stage, Year, Quarter, applicant;
"""
pydb.create_temp_table(create_averages_30_table,'averages_30')

In [ ]:
#Conditional Order - Quarterly, with applicant split - averages table

create_averages_31_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  applicant,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_nisi >= 0 THEN pet_to_nisi END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END) + MIN(CASE WHEN data_half = 2 THEN pet_to_nisi END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    new_case_type as "Case",
    CASE WHEN Stage = 'Petition' then 'Application'
         WHEN Stage = 'Decree Nisi' then 'Conditional Order'
         WHEN Stage = 'Decree Absolute' then 'Final Order'
         ELSE 'check' 
    END AS Stage ,
    CAST(NULL AS INT) AS Year,
    CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
    nfd_app_type as applicant,
    pet_to_nisi,
    NTILE(2) OVER (PARTITION BY new_case_type, stage, year, nfd_app_type 
                   ORDER BY pet_to_nisi) AS data_half

  FROM fcsq.divorce_fct_nisi
  
  WHERE law = 'New'
  
  ORDER BY new_case_type, stage, year, nfd_app_type)
  
GROUP BY "Case", Stage, Year, Quarter, applicant

ORDER BY "Case", Stage, Year, Quarter, applicant;
"""
pydb.create_temp_table(create_averages_31_table,'averages_31')

In [ ]:
#Conditional Order - Quarterly, without applicant split - averages table

create_averages_32_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  applicant,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_nisi >= 0 THEN pet_to_nisi END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END) + MIN(CASE WHEN data_half = 2 THEN pet_to_nisi END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    new_case_type as "Case",
    CASE WHEN Stage = 'Petition' then 'Application'
         WHEN Stage = 'Decree Nisi' then 'Conditional Order'
         WHEN Stage = 'Decree Absolute' then 'Final Order'
         ELSE 'check' 
    END AS Stage ,
    CAST(NULL AS INT) AS Year,
    CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
    'All' as applicant,
    pet_to_nisi,
    NTILE(2) OVER (PARTITION BY new_case_type, stage, year 
                   ORDER BY pet_to_nisi) AS data_half

  FROM fcsq.divorce_fct_nisi
  
  WHERE law = 'New'
  
  ORDER BY new_case_type, stage, year)
  
GROUP BY "Case", Stage, Year, Quarter, applicant

ORDER BY "Case", Stage, Year, Quarter, applicant;
"""
pydb.create_temp_table(create_averages_32_table,'averages_32')

In [ ]:
#Final order - Annual, with applicant split - averages table

create_averages_33_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  applicant,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_abs >= 0 THEN pet_to_abs END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_abs END) + MIN(CASE WHEN data_half = 2 THEN pet_to_abs END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_abs END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    new_case_type as "Case",
    CASE WHEN Stage = 'Petition' then 'Application'
         WHEN Stage = 'Decree Nisi' then 'Conditional Order'
         WHEN Stage = 'Decree Absolute' then 'Final Order'
         ELSE 'check' 
    END AS Stage ,
    Year,
    CAST(NULL AS VARCHAR) AS Quarter,
    nfd_app_type as applicant,
    pet_to_abs,
    NTILE(2) OVER (PARTITION BY new_case_type, stage, year, nfd_app_type 
                   ORDER BY pet_to_abs) AS data_half

  FROM fcsq.divorce_fct_abs
  
  WHERE law = 'New'
  
  ORDER BY new_case_type, stage, year, nfd_app_type)
  
GROUP BY "Case", Stage, Year, Quarter, applicant

ORDER BY "Case", Stage, Year, Quarter, applicant;
"""
pydb.create_temp_table(create_averages_33_table,'averages_33')

In [ ]:
#Final order - Annual, without applicant split - averages table

create_averages_34_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  applicant,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_abs >= 0 THEN pet_to_abs END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_abs END) + MIN(CASE WHEN data_half = 2 THEN pet_to_abs END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_abs END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    new_case_type as "Case",
    CASE WHEN Stage = 'Petition' then 'Application'
         WHEN Stage = 'Decree Nisi' then 'Conditional Order'
         WHEN Stage = 'Decree Absolute' then 'Final Order'
         ELSE 'check' 
    END AS Stage ,
    Year,
    CAST(NULL AS VARCHAR) AS Quarter,
    'All' as applicant,
    pet_to_abs,
    NTILE(2) OVER (PARTITION BY new_case_type, stage, year 
                   ORDER BY pet_to_abs) AS data_half

  FROM fcsq.divorce_fct_abs
  
  WHERE law = 'New'
  
  ORDER BY new_case_type, stage, year)
  
GROUP BY "Case", Stage, Year, Quarter, applicant

ORDER BY "Case", Stage, Year, Quarter, applicant;
"""
pydb.create_temp_table(create_averages_34_table,'averages_34')

In [ ]:
#Final order - Quarter, with applicant split - averages table

create_averages_35_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  applicant,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_abs >= 0 THEN pet_to_abs END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_abs END) + MIN(CASE WHEN data_half = 2 THEN pet_to_abs END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_abs END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    new_case_type as "Case",
    CASE WHEN Stage = 'Petition' then 'Application'
         WHEN Stage = 'Decree Nisi' then 'Conditional Order'
         WHEN Stage = 'Decree Absolute' then 'Final Order'
         ELSE 'check' 
    END AS Stage ,
    CAST(NULL AS INT) AS Year,
    CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
    nfd_app_type as applicant,
    pet_to_abs,
    NTILE(2) OVER (PARTITION BY new_case_type, stage, year, nfd_app_type 
                   ORDER BY pet_to_abs) AS data_half

  FROM fcsq.divorce_fct_abs
  
  WHERE law = 'New'
  
  ORDER BY new_case_type, stage, year, nfd_app_type)
  
GROUP BY "Case", Stage, Year, Quarter, applicant

ORDER BY "Case", Stage, Year, Quarter, applicant;
"""
pydb.create_temp_table(create_averages_35_table,'averages_35')

In [ ]:
#Final order - Quarter, without applicant split - averages table

create_averages_36_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  applicant,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_abs >= 0 THEN pet_to_abs END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_abs END) + MIN(CASE WHEN data_half = 2 THEN pet_to_abs END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_abs END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    new_case_type as "Case",
    CASE WHEN Stage = 'Petition' then 'Application'
         WHEN Stage = 'Decree Nisi' then 'Conditional Order'
         WHEN Stage = 'Decree Absolute' then 'Final Order'
         ELSE 'check' 
    END AS Stage ,
    CAST(NULL AS INT) AS Year,
    CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
    'All' as applicant,
    pet_to_abs,
    NTILE(2) OVER (PARTITION BY new_case_type, stage, year 
                   ORDER BY pet_to_abs) AS data_half

  FROM fcsq.divorce_fct_abs
  
  WHERE law = 'New'
  
  ORDER BY new_case_type, stage, year)
  
GROUP BY "Case", Stage, Year, Quarter, applicant

ORDER BY "Case", Stage, Year, Quarter, applicant;
"""
pydb.create_temp_table(create_averages_36_table,'averages_36')

In [ ]:
#Applications - Annual, with applicant split - Dissolution of marriage figures - averages table

create_averages_37_table =f"""
SELECT
  'Divorce and Civil Partnership' AS "Case",
  CASE WHEN Stage = 'Petition' then 'Application'
       WHEN Stage = 'Decree Nisi' then 'Conditional Order'
       WHEN Stage = 'Decree Absolute' then 'Final Order'
       ELSE 'check' 
  END AS Stage,
  Year,
  CAST(NULL AS VARCHAR) AS Quarter,
  nfd_app_type as applicant,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM fcsq.divorce_fct_petitions

WHERE law = 'New' AND Proceeding_type_code = 'D'

GROUP BY Stage, Year, nfd_app_type; 
"""
pydb.create_temp_table(create_averages_37_table,'averages_37')

In [ ]:
#Applications - Annual, without applicant split - Dissolution of marriage figures - averages table

create_averages_38_table =f"""
SELECT
  'Divorce and Civil Partnership' AS "Case",
  CASE WHEN Stage = 'Petition' then 'Application'
       WHEN Stage = 'Decree Nisi' then 'Conditional Order'
       WHEN Stage = 'Decree Absolute' then 'Final Order'
       ELSE 'check' 
  END AS Stage,
  Year,
  CAST(NULL AS VARCHAR) AS Quarter,
  'All' as applicant,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM fcsq.divorce_fct_petitions

WHERE law = 'New' AND Proceeding_type_code = 'D'

GROUP BY Stage, Year;
"""
pydb.create_temp_table(create_averages_38_table,'averages_38')

In [ ]:
 #Applications - Quarterly, with applicant split - Dissolution of marriage figures - averages table 

create_averages_39_table =f"""
SELECT
  'Divorce and Civil Partnership' AS "Case",
  CASE WHEN Stage = 'Petition' then 'Application'
       WHEN Stage = 'Decree Nisi' then 'Conditional Order'
       WHEN Stage = 'Decree Absolute' then 'Final Order'
       ELSE 'check' 
  END AS Stage,
  CAST(NULL AS INT) AS Year,
  CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
  nfd_app_type as applicant,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM fcsq.divorce_fct_petitions

WHERE law = 'New' AND Proceeding_type_code = 'D'

GROUP BY Stage, Year, Quarter, nfd_app_type; 
"""
pydb.create_temp_table(create_averages_39_table,'averages_39')

In [ ]:
#Applications - Quarterly, without applicant split - Dissolution of marriage figures - averages table

create_averages_40_table =f"""
SELECT
  'Divorce and Civil Partnership' AS "Case",
  CASE WHEN Stage = 'Petition' then 'Application'
       WHEN Stage = 'Decree Nisi' then 'Conditional Order'
       WHEN Stage = 'Decree Absolute' then 'Final Order'
       ELSE 'check' 
  END AS Stage,
  CAST(NULL AS INT) AS Year,
  CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
  'All' as applicant,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM fcsq.divorce_fct_petitions

WHERE law = 'New' AND Proceeding_type_code = 'D'

GROUP BY Stage, Year, Quarter;
"""
pydb.create_temp_table(create_averages_40_table,'averages_40')

In [ ]:
#Conditional order - Annual, with applicant split - Dissolution of marriage figures - averages table

create_averages_41_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  applicant,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_nisi >= 0 THEN pet_to_nisi END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END) + MIN(CASE WHEN data_half = 2 THEN pet_to_nisi END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    'Divorce and Civil Partnership' as "Case",
    CASE WHEN Stage = 'Petition' then 'Application'
         WHEN Stage = 'Decree Nisi' then 'Conditional Order'
         WHEN Stage = 'Decree Absolute' then 'Final Order'
         ELSE 'check' 
    END AS Stage ,
    Year,
    CAST(NULL AS VARCHAR) AS Quarter,
    nfd_app_type as applicant,
    pet_to_nisi,
    NTILE(2) OVER (PARTITION BY new_case_type, stage, year, nfd_app_type 
                   ORDER BY pet_to_nisi) AS data_half

  FROM fcsq.divorce_fct_nisi
  
  WHERE law = 'New' AND Proceeding_type_code = 'D'
  
  ORDER BY stage, year, nfd_app_type)
  
GROUP BY "Case", Stage, Year, Quarter, applicant

ORDER BY "Case", Stage, Year, Quarter, applicant;
"""
pydb.create_temp_table(create_averages_41_table,'averages_41')

In [ ]:
#Conditional order - Annual, without applicant split - Dissolution of marriage figures - averages table

create_averages_42_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  applicant,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_nisi >= 0 THEN pet_to_nisi END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END) + MIN(CASE WHEN data_half = 2 THEN pet_to_nisi END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    'Divorce and Civil Partnership' as "Case",
    CASE WHEN Stage = 'Petition' then 'Application'
         WHEN Stage = 'Decree Nisi' then 'Conditional Order'
         WHEN Stage = 'Decree Absolute' then 'Final Order'
         ELSE 'check' 
    END AS Stage ,
    Year,
    CAST(NULL AS VARCHAR) AS Quarter,
    'All' AS applicant,
    pet_to_nisi,
    NTILE(2) OVER (PARTITION BY new_case_type, stage, year 
                   ORDER BY pet_to_nisi) AS data_half

  FROM fcsq.divorce_fct_nisi
  
  WHERE law = 'New' AND Proceeding_type_code = 'D'
  
  ORDER BY stage, year)
  
GROUP BY "Case", Stage, Year, Quarter, applicant

ORDER BY "Case", Stage, Year, Quarter, applicant;
"""
pydb.create_temp_table(create_averages_42_table,'averages_42')

In [ ]:
#Conditional Order - Quarterly, with applicant split - Dissolution of marriage figures - averages table

create_averages_43_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  applicant,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_nisi >= 0 THEN pet_to_nisi END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END) + MIN(CASE WHEN data_half = 2 THEN pet_to_nisi END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    'Divorce and Civil Partnership' as "Case",
    CASE WHEN Stage = 'Petition' then 'Application'
         WHEN Stage = 'Decree Nisi' then 'Conditional Order'
         WHEN Stage = 'Decree Absolute' then 'Final Order'
         ELSE 'check' 
    END AS Stage ,
    CAST(NULL AS INT) AS Year,
    CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
    nfd_app_type as applicant,
    pet_to_nisi,
    NTILE(2) OVER (PARTITION BY new_case_type, stage, year, nfd_app_type 
                   ORDER BY pet_to_nisi) AS data_half

  FROM fcsq.divorce_fct_nisi
  
  WHERE law = 'New' AND Proceeding_type_code = 'D'
  
  ORDER BY stage, year, nfd_app_type)
  
GROUP BY "Case", Stage, Year, Quarter, applicant

ORDER BY "Case", Stage, Year, Quarter, applicant;
"""
pydb.create_temp_table(create_averages_43_table,'averages_43')

In [ ]:
#Conditional Order - Quarterly, without applicant split - Dissolution of marriage figures - averages table

create_averages_44_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  applicant,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_nisi >= 0 THEN pet_to_nisi END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END) + MIN(CASE WHEN data_half = 2 THEN pet_to_nisi END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_nisi END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    'Divorce and Civil Partnership' as "Case",
    CASE WHEN Stage = 'Petition' then 'Application'
         WHEN Stage = 'Decree Nisi' then 'Conditional Order'
         WHEN Stage = 'Decree Absolute' then 'Final Order'
         ELSE 'check' 
    END AS Stage ,
    CAST(NULL AS INT) AS Year,
    CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
    'All' as applicant,
    pet_to_nisi,
    NTILE(2) OVER (PARTITION BY new_case_type, stage, year 
                   ORDER BY pet_to_nisi) AS data_half

  FROM fcsq.divorce_fct_nisi
  
  WHERE law = 'New' AND Proceeding_type_code = 'D'
  
  ORDER BY stage, year)
  
GROUP BY "Case", Stage, Year, Quarter, applicant

ORDER BY "Case", Stage, Year, Quarter, applicant;
"""
pydb.create_temp_table(create_averages_44_table,'averages_44')

In [ ]:
#Final order - Annual, with applicant split - Dissolution of marriage figures - averages table

create_averages_45_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  applicant,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_abs >= 0 THEN pet_to_abs END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_abs END) + MIN(CASE WHEN data_half = 2 THEN pet_to_abs END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_abs END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    'Divorce and Civil Partnership' as "Case",
    CASE WHEN Stage = 'Petition' then 'Application'
         WHEN Stage = 'Decree Nisi' then 'Conditional Order'
         WHEN Stage = 'Decree Absolute' then 'Final Order'
         ELSE 'check' 
    END AS Stage ,
    Year,
    CAST(NULL AS VARCHAR) AS Quarter,
    nfd_app_type as applicant,
    pet_to_abs,
    NTILE(2) OVER (PARTITION BY new_case_type, stage, year, nfd_app_type 
                   ORDER BY pet_to_abs) AS data_half

  FROM fcsq.divorce_fct_abs
  
  WHERE law = 'New' AND Proceeding_type_code = 'D'
  
  ORDER BY stage, year, nfd_app_type)
  
GROUP BY "Case", Stage, Year, Quarter, applicant

ORDER BY "Case", Stage, Year, Quarter, applicant;
"""
pydb.create_temp_table(create_averages_45_table,'averages_45')

In [ ]:
#Final order - Annual, without applicant split - Dissolution of marriage figures - averages table

create_averages_46_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  applicant,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_abs >= 0 THEN pet_to_abs END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_abs END) + MIN(CASE WHEN data_half = 2 THEN pet_to_abs END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_abs END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    'Divorce and Civil Partnership' as "Case",
    CASE WHEN Stage = 'Petition' then 'Application'
         WHEN Stage = 'Decree Nisi' then 'Conditional Order'
         WHEN Stage = 'Decree Absolute' then 'Final Order'
         ELSE 'check' 
    END AS Stage ,
    Year,
    CAST(NULL AS VARCHAR) AS Quarter,
    'All' as applicant,
    pet_to_abs,
    NTILE(2) OVER (PARTITION BY new_case_type, stage, year 
                   ORDER BY pet_to_abs) AS data_half

  FROM fcsq.divorce_fct_abs
  
  WHERE law = 'New' AND Proceeding_type_code = 'D'
  
  ORDER BY stage, year)
  
GROUP BY "Case", Stage, Year, Quarter, applicant

ORDER BY "Case", Stage, Year, Quarter, applicant;
"""
pydb.create_temp_table(create_averages_46_table,'averages_46')

In [ ]:
#Final order - Quarter, with applicant split - Dissolution of marriage figures - averages table

create_averages_47_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  applicant,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_abs >= 0 THEN pet_to_abs END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_abs END) + MIN(CASE WHEN data_half = 2 THEN pet_to_abs END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_abs END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    'Divorce and Civil Partnership' as "Case",
    CASE WHEN Stage = 'Petition' then 'Application'
         WHEN Stage = 'Decree Nisi' then 'Conditional Order'
         WHEN Stage = 'Decree Absolute' then 'Final Order'
         ELSE 'check' 
    END AS Stage ,
    CAST(NULL AS INT) AS Year,
    CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
    nfd_app_type as applicant,
    pet_to_abs,
    NTILE(2) OVER (PARTITION BY new_case_type, stage, year, nfd_app_type 
                   ORDER BY pet_to_abs) AS data_half

  FROM fcsq.divorce_fct_abs
  
  WHERE law = 'New' AND Proceeding_type_code = 'D'
  
  ORDER BY stage, year, nfd_app_type)
  
GROUP BY "Case", Stage, Year, Quarter, applicant

ORDER BY "Case", Stage, Year, Quarter, applicant;
"""
pydb.create_temp_table(create_averages_47_table,'averages_47')

In [ ]:
#Final order - Quarter, without applicant split - Dissolution of marriage figures - averages table

create_averages_48_table =f"""
SELECT 
  "Case",
  Stage,
  Year,
  Quarter,
  applicant,
  count(*) as count,
  ROUND(AVG(CASE WHEN pet_to_abs >= 0 THEN pet_to_abs END)/7,3) AS mean_weeks,
  ROUND((CASE WHEN COUNT(*) % 2 = 0
              THEN (MAX(CASE WHEN data_half = 1 THEN pet_to_abs END) + MIN(CASE WHEN data_half = 2 THEN pet_to_abs END)) /2.0 
              ELSE MAX(CASE WHEN data_half = 1 THEN pet_to_abs END)
         END) / 7, 3) AS median_weeks 
         
FROM (
  SELECT
    'Divorce and Civil Partnership' as "Case",
    CASE WHEN Stage = 'Petition' then 'Application'
         WHEN Stage = 'Decree Nisi' then 'Conditional Order'
         WHEN Stage = 'Decree Absolute' then 'Final Order'
         ELSE 'check' 
    END AS Stage ,
    CAST(NULL AS INT) AS Year,
    CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
    'All' as applicant,
    pet_to_abs,
    NTILE(2) OVER (PARTITION BY new_case_type, stage, year 
                   ORDER BY pet_to_abs) AS data_half

  FROM fcsq.divorce_fct_abs
  
  WHERE law = 'New' AND Proceeding_type_code = 'D'
  
  ORDER BY stage, year)
  
GROUP BY "Case", Stage, Year, Quarter, applicant

ORDER BY "Case", Stage, Year, Quarter, applicant;
"""
pydb.create_temp_table(create_averages_48_table,'averages_48')

In [ ]:
drop_new_divorce_timeliness = "DROP TABLE IF EXISTS fcsq.new_divorce_timeliness"
pydb.start_query_execution_and_wait(drop_new_divorce_timeliness)
bucket.objects.filter(Prefix="fcsq_processing/Divorce/new_divorce_timeliness").delete();


In [ ]:
#Creating timeliness table combining all the new law average tables together 

create_new_divorce_timeliness_table =f"""
CREATE TABLE IF NOT EXISTS fcsq.new_divorce_timeliness
WITH (format = 'PARQUET', external_location = 's3://alpha-family-data/fcsq_processing/Divorce/new_divorce_timeliness') AS
SELECT * FROM __temp__.averages_25
UNION ALL
SELECT * FROM __temp__.averages_26
UNION ALL
SELECT * FROM __temp__.averages_27
UNION ALL
SELECT * FROM __temp__.averages_28
UNION ALL
SELECT * FROM __temp__.averages_29
UNION ALL
SELECT * FROM __temp__.averages_30
UNION ALL
SELECT * FROM __temp__.averages_31
UNION ALL
SELECT * FROM __temp__.averages_32
UNION ALL
SELECT * FROM __temp__.averages_33
UNION ALL
SELECT * FROM __temp__.averages_34
UNION ALL
SELECT * FROM __temp__.averages_35
UNION ALL
SELECT * FROM __temp__.averages_36
UNION ALL
SELECT * FROM __temp__.averages_37
UNION ALL
SELECT * FROM __temp__.averages_38
UNION ALL
SELECT * FROM __temp__.averages_39
UNION ALL
SELECT * FROM __temp__.averages_40
UNION ALL
SELECT * FROM __temp__.averages_41
UNION ALL
SELECT * FROM __temp__.averages_42
UNION ALL
SELECT * FROM __temp__.averages_43
UNION ALL
SELECT * FROM __temp__.averages_44
UNION ALL
SELECT * FROM __temp__.averages_45
UNION ALL
SELECT * FROM __temp__.averages_46
UNION ALL
SELECT * FROM __temp__.averages_47
UNION ALL
SELECT * FROM __temp__.averages_48
;
"""
pydb.start_query_execution_and_wait(create_new_divorce_timeliness_table)

In [ ]:
#test = pydb.read_sql_query("SELECT * from fcsq.new_divorce_timeliness LIMIT 10")
#test

In [ ]:
#pydb.delete_table_and_data(database="__temp__", table="new_divorce_timeliness")

In [ ]:
create_new_divorce_timeliness_gender_table =f"""

/* Selecting the original table without gender information*/

SELECT
 "Case", 
 Stage,
 Year, 
 Quarter, 
 Applicant, 
 'All' as main_applicant_gender, 
 'All' as other_applicant_gender,
 Count,
 Mean_weeks,
 Median_weeks

FROM fcsq.new_divorce_timeliness

UNION ALL

/*Adding gender info for Joint cases - Annually*/

SELECT
  New_case_type AS "Case",
  CASE WHEN Stage = 'Petition' then 'Application'
       WHEN Stage = 'Decree Nisi' then 'Conditional Order'
       WHEN Stage = 'Decree Absolute' then 'Final Order'
       ELSE Stage 
  END AS Stage,
  Year,
  CAST(NULL AS VARCHAR) AS Quarter,
  nfd_app_type as applicant,
  CASE WHEN petitioners_gender = 'check' THEN 'Unknown'
       ELSE petitioners_gender 
  END AS main_applicant_gender,
  CASE WHEN respondents_gender = 'check' THEN 'Unknown'
       ELSE respondents_gender 
  END AS other_applicant_gender,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks
    
FROM fcsq.divorce_fct_petitions

WHERE law = 'New'AND nfd_app_type != 'Sole'

GROUP BY
  New_case_type,
  Stage,
  Petitioners_gender,
  Respondents_gender,
  Year,
  nfd_app_type 


UNION ALL

/*Adding gender info for Joint cases - Quarterly*/

SELECT
  New_case_type AS "Case",
  CASE WHEN Stage = 'Petition' then 'Application'
       WHEN Stage = 'Decree Nisi' then 'Conditional Order'
       WHEN Stage = 'Decree Absolute' then 'Final Order'
       ELSE Stage 
  END AS Stage,
  CAST(NULL AS INT) AS Year,
  CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
  nfd_app_type as applicant,
  CASE WHEN petitioners_gender = 'check' THEN 'Unknown'
       ELSE petitioners_gender 
  END as main_applicant_gender,
  CASE WHEN respondents_gender = 'check' THEN 'Unknown'
       ELSE respondents_gender 
  END as other_applicant_gender,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks
    
FROM fcsq.divorce_fct_petitions

WHERE law = 'New' and nfd_app_type != 'Sole'

GROUP BY
  New_case_type,
  Stage,
  Petitioners_gender,
  Respondents_gender,
  Year,
  Quarter,
  nfd_app_type 

UNION ALL

/*Adding gender info for sole cases - Annually*/

SELECT
  New_case_type AS "Case",
  CASE WHEN Stage = 'Petition' then 'Application'
       WHEN Stage = 'Decree Nisi' then 'Conditional Order'
       WHEN Stage = 'Decree Absolute' then 'Final Order'
       ELSE Stage 
  END AS Stage,
  Year,
  CAST(NULL AS VARCHAR) AS Quarter,
  nfd_app_type as applicant,
  CASE WHEN petitioners_gender = 'check' THEN 'Unknown'
       ELSE petitioners_gender 
  END as main_applicant_gender,
  'N/A' as other_applicant_gender,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM fcsq.divorce_fct_petitions

WHERE law = 'New' AND nfd_app_type = 'Sole'

GROUP BY
  New_case_type,
  Stage,
  Petitioners_gender,
  Year,
  nfd_app_type 


UNION ALL

/*Adding gender info for Sole cases - Quarterly*/

SELECT
  New_case_type AS "Case",
  CASE WHEN Stage = 'Petition' then 'Application'
       WHEN Stage = 'Decree Nisi' then 'Conditional Order'
       WHEN Stage = 'Decree Absolute' then 'Final Order'
       ELSE Stage 
  END AS Stage,
  CAST(NULL AS INT) AS Year,
  CONCAT(CAST(Year AS VARCHAR), ' Q', CAST(Quarter AS VARCHAR)) AS Quarter,
  nfd_app_type as applicant,
  CASE WHEN petitioners_gender = 'check' THEN 'Unknown'
       ELSE petitioners_gender 
  END as main_applicant_gender,
  'N/A' as other_applicant_gender,
  count(*) as count,
  CAST(NULL AS INT) AS mean_weeks,
  CAST(NULL AS INT) AS median_weeks

FROM fcsq.divorce_fct_petitions

WHERE law = 'New' AND nfd_app_type = 'Sole'

GROUP BY
  New_case_type,
  Stage,
  Petitioners_gender,
  Year,
  Quarter,
  nfd_app_type;
  
  """
pydb.create_temp_table(create_new_divorce_timeliness_gender_table,'new_divorce_timeliness_gender')
